In [1]:
# Class label csv path
labels_csv_path = 'datasets/kinetics_400_labels.csv'

In [2]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip
import sounddevice as sd
import numpy as np
from torch.nn import functional as F
from scipy.io.wavfile import write
import time
import threading
import queue
from IPython.display import display, clear_output

In [3]:
import serial

# ls /dev/tty.*
ser = serial.Serial('/dev/tty.usbmodem11201', 9600)

# while True:
#     if ser.in_waiting > 0:
#         line = ser.readline().decode('utf-8').strip()
#         print(line)

In [4]:
import csv

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)

    for row in csv_reader:
        class_id, class_name = row
        label2id[class_name] = class_id
        id2label[class_id] = class_name

class_labels = list(label2id.keys())
class_labels.append("glass breaking")

# print(f"{len(class_labels)} Unique classes: {class_labels}.")

In [5]:
window_size = 10  # seconds
sample_rate = 16000  # Hz

# Open the serial port
ser = serial.Serial('/dev/tty.usbmodem11201', 9600)

print("Loading model...")
# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version='2023', use_cuda=False)

print("Extracting text embeddings...")
# Extract text embeddings
text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}" for c in class_labels])

print("Listening...")

while True:    
    # Read a chunk of audio data from the serial port
    audio_data = ser.read(window_size * sample_rate * 2)  # 2 bytes per sample

    # Convert the audio data to an array of samples
    recording = np.frombuffer(audio_data, dtype=np.int16)

    # Save the recording to a temporary file
    temp_filename = "temp_recording.wav"
    write(temp_filename, sample_rate, recording)

    # Extract audio embeddings
    audio_embeddings = clap_model.get_audio_embeddings([temp_filename])
    # Compute similarity between audio and text embeddings
    similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

    similarity = F.softmax(similarities, dim=1)
    values, indices = similarity[0].topk(5)
    
    # Check if "glass breaking" is in the top five predictions
    things_to_detect = ["glass breaking", "using computer"]
    detected = any(class_labels[index] in things_to_detect for index in indices)
    
    print("Top predictions:\n" + "\n".join(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%" for value, index in zip(values, indices)) + "\n")
    
    if detected:
        filename = f"recording_{int(time.time())}.wav"
        os.rename(temp_filename, filename)
        print(f"Alert! Glass breaking detected in {filename}")
    else:
        os.remove(temp_filename) 


Loading model...
Extracting text embeddings...
Listening...
Top predictions:
         smoking: 24.53%
       tying tie: 8.01%
  massaging legs: 5.01%
       bandaging: 4.69%
sign language interpreting: 3.88%

Top predictions:
  glass breaking: 32.11%
  sword fighting: 8.91%
hitting baseball: 8.01%
   tossing salad: 7.71%
    disc golfing: 7.31%

Alert! Glass breaking detected in recording_1713753886.wav
Top predictions:
  drinking shots: 19.43%
         smoking: 14.02%
   drinking beer: 11.13%
    tasting beer: 4.36%
       tying tie: 3.89%

Top predictions:
  massaging legs: 16.02%
       tying tie: 12.08%
       bandaging: 11.18%
massaging person's head: 9.39%
     petting cat: 3.32%

Top predictions:
       tying tie: 16.70%
   hoverboarding: 13.18%
     petting cat: 10.89%
       bandaging: 5.69%
  massaging legs: 5.39%

Top predictions:
  glass breaking: 49.64%
    disc golfing: 22.99%
      barbequing: 2.07%
     ice fishing: 2.05%
   tossing salad: 2.00%

Alert! Glass breaking d

KeyboardInterrupt: 

In [6]:
ser.close()